In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import parse
import json
import copy

In [33]:
def parse_transcript(filename):
    spaces_regex = re.compile("^(\s*).*")
    location_regex = re.compile("^\s*(INT\.|EXT\.)")
    line_list = []
    options = [0] * 100
    opt = []
    transcript = []

    text_file = open(filename, "r")
    lines = text_file.readlines()
    for l in lines:
        li = l.strip(' \n\t\r')
        if li != "":
            spmatch = spaces_regex.search(l)
            spaces_number = len(spmatch.group(1))
            options[spaces_number] += 1
            line_list.append((li, spaces_number))
    for i in range(0, 50):
        if options[i] >= 20:
            opt.append(i)
    print(opt)
    text_file.close()

    speaker = ""
    utterance = ""

    for l in lines:
        li = l.strip(' \n\t\r')
        if li != "":
            spmatch = spaces_regex.search(l)
            spaces_number = len(spmatch.group(1))
            if spaces_number == opt[2]:
                if utterance != "" and speaker != "":
                    transcript.append(
                        {'speaker': speaker, 'utterance': utterance.strip()})
                    utterance = ""
                speaker = re.sub(r'\([^()]*\)', '', li).strip(' \n\t\r')
            elif spaces_number == opt[1]:
                utterance += " " + li
            else:
                if utterance != "" and speaker != "":
                    transcript.append(
                        {'speaker': speaker, 'utterance': utterance.strip()})
                    utterance = ""
                    speaker = ""

    return transcript

In [37]:
SCRIPT_PATH = "./data/scripts/V-for-Vendetta.txt"
MOVIE_NAME = "V for Vendetta"
parsed_transcript = (parse_transcript(SCRIPT_PATH))

[10, 20, 30]


In [38]:
def make_line_dict(transcript):
    line_dict = dict()

    for i in np.arange(len(parsed_transcript)):
        speaker = parsed_transcript[i]['speaker']
        line = parsed_transcript[i]['utterance']
        if speaker in line_dict.keys():
            line_dict[speaker] += [line]
        else:
            line_dict[speaker] = [line]
            
    return line_dict

In [39]:
credits = pd.read_csv('./data/tmdb_5000_credits.csv')

In [118]:
movie_titles = np.array(credits["title"].tolist())
movie_num = np.where(movie_titles == "V for Vendetta")
movie_cast = np.array(credits["cast"].tolist())[movie_num][0]
line_dict = make_line_dict(parsed_transcript)

In [170]:
movie_char_list = json.loads(movie_cast)
print(len(line_dict))
new_dict = dict()
movie_char_list_copy = copy.deepcopy(movie_char_list)


for speaker in line_dict.keys():
    speaker_dist = list()
    for d in movie_char_list_copy:
        if speaker.lower() == d['character'].lower() or speaker.lower() in ([x.lower() for x in d['character'].split(' ')]):            
            new_dict[d['character']] = speaker, d['name'], d['gender']
            movie_char_list_copy.remove(d)
            break

59


In [186]:
ethnicities = pd.read_csv('./data/ethnicelebs.csv')
print(ethnicities)
            
for character in new_dict.keys():
    actor = '-'.join(new_dict[character][1].lower().split(' '))


                       aaliyah  \
0                aaron-eckhart   
1                   aaron-paul   
2               aaron-stanford   
3         aaron-taylor-johnson   
4                  aaron-tveit   
5                abbie-cornish   
6                 abby-elliott   
7              abigail-breslin   
8              abigail-spencer   
9              abraham-benrubi   
10                  adam-brody   
11                 adam-devine   
12                 adam-driver   
13               adam-goldberg   
14                 adam-levine   
15                adam-sandler   
16                  adam-scott   
17                   adam-west   
18    adewale-akinnuoye-agbaje   
19                adhir-kalyan   
20              adrian-bellani   
21              adrian-grenier   
22               adrian-pasdar   
23             adriana-barraza   
24            adrianne-palicki   
25            adriano-giannini   
26                adrien-brody   
27            adrienne-barbeau   
28            